In [1]:
# Code written by Aiden James
# Create 2 folders in the base directory labeled "input" and "output"
# Input files go into the folder labeled as such
# Files must be PNG or JPG and bitmap format


from PIL import Image
import numpy as np
import cv2
import os
import sympy
from sympy import Matrix
import copy
from matplotlib import pyplot as plt
from skimage.io import imread
from skimage.color import rgb2gray
from scipy import signal as sig
from scipy import ndimage as ndi

edgeKernelThreshold = 10 # The threshold for the edge kernel. This does not need to change much. In the future we would like to automatically determine it
edgeThresholdPower = -4

edgeKernel = np.array([
                      [0.0, -1.0, 0.0],
                      [-1.0, 4.0, -1.0],
                      [0.0, -1.0, 0.0]
                      ])
blurKernel = np.array([
                      [1/16,2/16,1/16],
                      [2/16,1/4,2/16],
                      [1/16,2/16,1/16],
                      ])
sharpenKernel = np.array([
                      [0,-1,0],
                      [-1,5,-1],
                      [0,-1,0],
                      ])


def getAverageColor(clr):
  clr = [np.abs(element) for element in clr]
  return (clr[0] + clr[1] + clr[2])/3


def getStandardDevOfColors(image, width, height):
  sum = np.array([0,0,0]) 
  for i in range(width):
    for j in range(height):
      sum = np.add(sum, np.array(image[i][j]))

# compute the standard devivation of the image
  sum = np.divide(sum, width * height)

  stdev = np.array([0,0,0])
  for i in range(width):
    for j in range(height):
        difference = np.subtract(sum, image[i][j])  #[(a - b) ** 2 for a, b in zip(sum, image[i][j])]
        difference = np.power(difference, 2)
        stdev = np.add(stdev, difference)
  stdev = np.divide(stdev, width * height)
  stdev = np.power(stdev, 1/2)
  return stdev


def gradient_x(imggray):
  ##Sobel operator kernels.
  kernel_x = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]])
  return sig.convolve2d(imggray, kernel_x, mode='same')
def gradient_y(imggray):
  kernel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
  return sig.convolve2d(imggray, kernel_y, mode='same')


############ MAIN CODE START ############

inputDir = "/content/input/"
outputDir = "/content/output/"

for filename in os.listdir(inputDir):
  f = os.path.join(inputDir, filename)

  if(os.path.isfile(f)):
    
    img = cv2.imread(f)
    noiseMatrix = np.array(copy.deepcopy(img)) 

    #matrix, mean, stdDev
    cv2.randn(noiseMatrix, (0,0,0), (20,20,20))  
    
    # grayscale image
    graySrc = rgb2gray(img)

    # I values for pixels
    Ix = gradient_x(graySrc)
    Iy = gradient_y(graySrc)
    
    # values of M matrix; the matrix never needs to be made to calculate the determinant and trace
    # Gaussian blur W 
    # Sigma adjusts gaussian blur

    IxIx = ndi.gaussian_filter(Ix**2, sigma=1)
    IxIy = ndi.gaussian_filter(Iy*Ix, sigma=1)
    IyIy = ndi.gaussian_filter(Iy**2, sigma=1)

    #determinant
    detA = IxIx * IyIy - IxIy ** 2

    #trace
    traceA = IxIx + IyIy

    k = 0.05
    r = detA - k * traceA ** 2
    img = np.add(img, noiseMatrix)

    # for use in seeing the edge value
    edgeImg = copy.deepcopy(img)
    edgeImg = cv2.filter2D(edgeImg, -1, edgeKernel)


    dst = copy.deepcopy(img)

    #uncomment the below line to output the file with noise applied
    #cv2.imwrite(os.path.join(outputDir, filename.replace(".png","") + "_CV" + ".png"), dst)

    # get size of image
    width, height, depth = img.shape


    # this is the threshold obtained from the average edge kernel value
    edgeThreshold = 0
    for row in r: #changed from img to r to check the average r score
      for clr in row:
        edgeThreshold +=clr
    edgeThreshold /= width * height 


    print(filename)
    print(edgeThreshold)
    #edgeThreshold = (-(np.abs(edgeThreshold)) ** -.5) +7
    #print(edgeThreshold)
    #edgeThreshold =  np.abs(edgeThreshold) / 35
    edgeThreshold = 10 ** edgeThresholdPower


    binaryMatrix = []
    for i in range(width):
      binaryVector = []
      for j in range(height):
        if np.abs(r[i][j]) > edgeThreshold and getAverageColor(edgeImg[i][j]) >= edgeKernelThreshold:
          binaryVector.append(1)
        else:
          binaryVector.append(0)
      binaryMatrix.append(binaryVector)
  
    # This part is just for displaying the output matrix. Ideally, you would only need the data to be a matrix of 1s and 0s. However,
    # to be viewed by humans we multiply it by 255 so it displays as black and white pixels on an image.
    whiteImageData = np.multiply(binaryMatrix, 255)

    cv2.imwrite(os.path.join(outputDir, filename), whiteImageData)
   
    
   




HybridImage.png
-0.002615212516612659


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: RuntimeWarning: overflow encountered in ubyte_scalars
